In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import time
from numpy import expand_dims


# Đọc DataFrame 'transaction'
transaction_df = pd.read_csv('../balanced_transaction-from-to-prefix-22-KLTN-newtime.csv')
print(transaction_df.shape)

(47463, 48)


In [2]:
# loading the lstm model
lstm_model = tf.keras.models.load_model('./Result-ML/final/lstm-100')
sgan_model = tf.keras.models.load_model('./Result-ML/final/sgan/100/disc_sup_0010.h5')

# Tiền xử lý dữ liệu
replace_nan = transaction_df.replace(np.nan, -1)

X = replace_nan.drop(columns=['hash','from_address','to_address','label'], axis=1).values

##### reshape SGAN #####
# Padding giá trị -2 để có số lượng đặc trưng là bội số của 49
num_padding = 49 - X.shape[1] % 49
X_padded = np.pad(X, ((0, 0), (0, num_padding)), mode='constant', constant_values=-2)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(X_padded)
X_reshaped_sgan = normalized_data.reshape(-1, 7, 7)
X_reshaped_sgan = expand_dims(X_reshaped_sgan, axis=-1)

##### reshape LSTM #####
# Padding giá trị -2 để có số lượng đặc trưng là bội số của 49
num_padding = 900 - X.shape[1] % 900
X_padded = np.pad(X, ((0, 0), (0, num_padding)), mode='constant', constant_values=-2)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(X_padded)
X_reshaped_lstm = normalized_data.reshape(-1, 30, 30)
X_reshaped_lstm = expand_dims(X_reshaped_lstm, axis=-1)

#dự đoán
y_pred_sgan=sgan_model.predict(X_reshaped_sgan)
y_pred_sgan=np.argmax(y_pred_sgan,axis=1)
y_pred_lstm=lstm_model.predict(X_reshaped_lstm)
y_pred_lstm=np.argmax(y_pred_lstm,axis=1)

# Kiểm tra xem số lượng dự đoán có khớp với số hàng trong transaction_df
if len(y_pred_lstm) == len(transaction_df) and len(y_pred_sgan) == len(transaction_df):
    # Thêm cột mới vào transaction_df
    transaction_df['lstm_prediction'] = y_pred_lstm
    transaction_df['sgan_prediction'] = y_pred_sgan
else:
    print("Số lượng dự đoán không khớp với số hàng trong DataFrame. Hãy kiểm tra lại.")

# Hiển thị một vài hàng đầu tiên để kiểm tra
print(transaction_df[['hash', 'label', 'lstm_prediction', 'sgan_prediction']].head())

# # Lưu DataFrame đã cập nhật (tùy chọn)
# transaction_df.to_csv('updated_transaction_df.csv', index=False)



1484/1484 [==============================] - 15s 10ms/step
                                                hash  label  lstm_prediction   
0  0x1d8dc0228d0ecc674c00dbc8c02dfc31e235ee1f2577...      1                1  \
1  0xdf3a1a30c2914b306dd8cb45b5227232daa29f809bfe...      0                0   
2  0xc4120db5c02a21116ad9e9dabc9dafa6ef2ef6e6e4e3...      0                0   
3  0xbe7c8d401d352a3b26364c02e2771d8869cd3c2e4269...      0                0   
4  0xa0a022845546742aa76ceb49b6b77b55df373c9f6467...      1                1   

   sgan_prediction  
0                1  
1                0  
2                0  
3                0  
4                1  


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Giả sử transaction_df đã được tiền xử lý và có các cột dự đoán của LSTM và SGAN

# 1. Chuẩn bị dữ liệu
X = transaction_df.drop(['hash', 'from_address', 'to_address', 'label', 'lstm_prediction', 'sgan_prediction'], axis=1)
y = transaction_df['label']

X = X.replace(np.nan, -1)

# Chia dữ liệu thành tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Lấy dự đoán từ LSTM và SGAN cho tập train và test
lstm_train_pred = transaction_df.loc[X_train.index, 'lstm_prediction']
sgan_train_pred = transaction_df.loc[X_train.index, 'sgan_prediction']
lstm_test_pred = transaction_df.loc[X_test.index, 'lstm_prediction']
sgan_test_pred = transaction_df.loc[X_test.index, 'sgan_prediction']

np.random.seed(42)
# 3. Tạo một tập dữ liệu nhỏ và cân bằng để huấn luyện ####### version 1
pos_samples = X_train[y_train == 1].sample(n=50, random_state=77)
neg_samples = X_train[y_train == 0].sample(n=50, random_state=77)
balanced_X_train = pd.concat([pos_samples, neg_samples])

pos_samples_lstm = lstm_train_pred[pos_samples.index]
neg_samples_lstm = lstm_train_pred[neg_samples.index]
pos_samples_sgan = sgan_train_pred[pos_samples.index]
neg_samples_sgan = sgan_train_pred[neg_samples.index]

balanced_lstm_pred = pd.concat([pos_samples_lstm, neg_samples_lstm])
balanced_sgan_pred = pd.concat([pos_samples_sgan, neg_samples_sgan])

balanced_y_train = pd.Series([1]*50 + [0]*50, index=balanced_X_train.index)

# 4. Tạo một mô hình meta-learner (ví dụ: Random Forest)
meta_features_train = np.column_stack((scaler.transform(balanced_X_train), balanced_lstm_pred, balanced_sgan_pred))
print(meta_features_train.shape)
meta_features_test = np.column_stack((X_test_scaled, lstm_test_pred, sgan_test_pred))
print(meta_features_test.shape)

# Random Forest
meta_learner = RandomForestClassifier(n_estimators=5000, random_state=42)
meta_learner.fit(meta_features_train, balanced_y_train)

# SVM
# meta_learner = SVC(kernel='rbf', random_state=42)
# meta_learner.fit(meta_features_train, balanced_y_train)

# KNN
# meta_learner = KNeighborsClassifier(n_neighbors=5)
# meta_learner.fit(meta_features_train, balanced_y_train)

# Logistic Regression
# meta_learner = LogisticRegression(random_state=42, max_iter=1000)
# meta_learner.fit(meta_features_train, balanced_y_train)

# Decision Tree
# meta_learner = DecisionTreeClassifier(random_state=42, max_depth=5)
# meta_learner.fit(meta_features_train, balanced_y_train)

# Naive Bayes
# meta_learner = GaussianNB()
# meta_learner.fit(meta_features_train, balanced_y_train)

# MLP
# meta_learner = MLPClassifier(
#     hidden_layer_sizes=(100, 50),  # Hai hidden layers với 100 và 50 neurons
#     activation='relu',
#     solver='adam',
#     alpha=0.0001,
#     max_iter=1000,
#     random_state=42
# )
# meta_learner.fit(meta_features_train, balanced_y_train)

# 5. Dự đoán sử dụng meta-learner
y_pred = meta_learner.predict(meta_features_test)

# 6. Đánh giá kết quả
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
tn, fp, fn, tp = conf_matrix.ravel()
fpr = fp / (fp + tn)
print(f"Dương tính giả (false positive rate): {fpr:.4f}")

# 7. Phân tích tầm quan trọng của các đặc trưng
# feature_importance = meta_learner.feature_importances_
# feature_names = list(X.columns) + ['lstm_prediction', 'sgan_prediction']
# importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importance})
# importance_df = importance_df.sort_values('importance', ascending=False)
# print("\nFeature Importance:")
# print(importance_df.head(10))

# 8. Áp dụng mô hình cho toàn bộ dữ liệu
# all_features = np.column_stack((scaler.transform(X), transaction_df['lstm_prediction'], transaction_df['sgan_prediction']))
# final_predictions = meta_learner.predict(all_features)

# Thêm dự đoán cuối cùng vào DataFrame gốc
# transaction_df['final_prediction'] = final_predictions

# Lưu kết quả
# transaction_df.to_csv('transaction_df_with_meta_predictions_balanced.csv', index=False)

(100, 46)
(9493, 46)
Accuracy: 0.9310
Precision: 0.9058
Recall: 0.9487
F1-score: 0.9268
Confusion Matrix:
[[4694  431]
 [ 224 4144]]
Dương tính giả (false positive rate): 0.0841


In [5]:
# Xử lý cho SGAN
num_padding_sgan = 49 - X_test.shape[1] % 49
X_padded_sgan = np.pad(X_test, ((0, 0), (0, num_padding_sgan)), mode='constant', constant_values=-2)
scaler_sgan = MinMaxScaler()
X_normalized_sgan = scaler_sgan.fit_transform(X_padded_sgan)
X_reshaped_sgan = X_normalized_sgan.reshape(-1, 7, 7)
X_reshaped_sgan = expand_dims(X_reshaped_sgan, axis=-1)

# Dự đoán
y_pred_sgan = sgan_model.predict(X_reshaped_sgan)
y_pred_sgan = np.argmax(y_pred_sgan, axis=1)

# Đánh giá kết quả SGAN
accuracy = accuracy_score(y_test, y_pred_sgan)
precision = precision_score(y_test, y_pred_sgan)
recall = recall_score(y_test, y_pred_sgan)
f1 = f1_score(y_test, y_pred_sgan)
conf_matrix = confusion_matrix(y_test, y_pred_sgan)
print("####### SGAN #######")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
tn, fp, fn, tp = conf_matrix.ravel()
fpr = fp / (fp + tn)
print(f"False Positive Rate: {fpr:.4f}")

297/297 [==============================] - 0s 897us/step
####### SGAN #######
Accuracy: 0.9288
Precision: 0.8941
Recall: 0.9588
F1-score: 0.9253
Confusion Matrix:
[[4629  496]
 [ 180 4188]]
False Positive Rate: 0.0968


In [5]:
import joblib

# Lưu meta-learner model
joblib.dump(meta_learner, './Result-ML/final/meta-learner-rf-5000.joblib')

['./Result-ML/final/meta-learner-rf-5000.joblib']